# Validation of find_attitude

This notebook run pseudo-random tests of the find_attitude algorithm covering many
attitudes, star inputs, and potential pitch or estimate attitude constraints.

Summary:
- Two-star solutions are dubious with ~30% being wrong.
- Using pitch and attitude constraints successfully limits the solution space.
- Even with no constraints, an attitude is successfully found with at least 4 stars.

In [1]:
import pickle
import sys
import traceback
from pathlib import Path

sys.path.insert(0, "/Users/aldcroft/git/ska_sun")
sys.path.insert(0, "/Users/aldcroft/git/ska_helpers")

import astropy.units as u
import numpy as np
import ska_sun
from cxotime import CxoTime
from Quaternion import Quat
from ska_helpers.utils import random_radec_in_cone
from tqdm import tqdm

import find_attitude as fa
import find_attitude.find_attitude as fafa
from find_attitude.tests import test_find_attitude

In [2]:
# Cache inputs to tests that fail
FAILS = []

In [3]:
def get_random_attitude(constraints: fa.Constraints) -> Quat:
    """
    Generates a random attitude based on the given constraints.

    Parameters
    ----------
    constraints : fa.Constraints
        The constraints for generating the attitude.

    Returns
    -------
    Quat
        Randomly generated attitude.

    """
    date = constraints.date

    # Back off from constraint off_nom_roll_max by 0.5 degrees to avoid randomly
    # generating an attitude that is too close to the constraint.
    onrm = max(0.0, constraints.off_nom_roll_max - 0.5)
    off_nom_roll = np.random.uniform(-onrm, onrm)

    if constraints.att is not None:
        att0 = Quat(constraints.att)
        # Back off from constraint att_err by 0.5 degrees (same as off_nom_roll).
        att_err = max(0.0, constraints.att_err - 0.5)
        ra, dec = random_radec_in_cone(att0.ra, att0.dec, att_err)
        roll0 = ska_sun.nominal_roll(ra, dec, time=date)
        roll = roll0 + off_nom_roll
        att = Quat([ra, dec, roll])

    elif constraints.pitch is not None:
        pitch_err = max(0.0, constraints.pitch_err - 0.5)
        d_pitch = np.random.uniform(pitch_err, pitch_err)
        pitch = constraints.pitch + d_pitch
        yaw = np.random.uniform(0, 360)
        att = ska_sun.get_att_for_sun_pitch_yaw(
            pitch, yaw, time=date, off_nom_roll=off_nom_roll
        )

    else:
        ra = np.random.uniform(0, 360)
        dec = np.rad2deg(np.arccos(np.random.uniform(-1, 1))) - 90
        roll = ska_sun.nominal_roll(ra, dec, time=date) + off_nom_roll
        att = Quat([ra, dec, roll])

    return att

In [4]:
def test_random_all_sky(
    sigma_1axis=1.0,
    sigma_mag=0.2,
    brightest=True,
    constraints=None,
    tolerance=2.5,
    min_stars=4,
    max_stars=8,
    log_level="WARNING",
    sherpa_log_level="WARNING",
    save_fails=False,
):
    if constraints is None:
        constraints = fa.Constraints(off_nom_roll_max=20, date="2025:001")

    n_stars = np.random.randint(min_stars, max_stars + 1)

    while True:
        att = get_random_attitude(constraints)
        stars = test_find_attitude.get_stars(
            att.ra,
            att.dec,
            att.roll,
            sigma_1axis=sigma_1axis,
            sigma_mag=sigma_mag,
            brightest=brightest,
            date=constraints.date,
        )
        if len(stars) >= n_stars:
            break

    stars = stars[:n_stars]
    solutions = []
    try:
        solutions = fa.find_attitude_solutions(
            stars,
            tolerance=tolerance,
            constraints=constraints,
            log_level=log_level,
            sherpa_log_level=sherpa_log_level,
        )

        for solution in solutions:
            solution["att"] = att

        test_find_attitude.check_output(solutions, stars, att.ra, att.dec, att.roll)
    except Exception:
        now = CxoTime.now().isot
        tb_str = traceback.format_exc()
        failed_test = {
            "date_run": now,
            "stars": stars,
            "tolerance": tolerance,
            "solutions": solutions,
            "att": att,
            "constraints": constraints,
            "traceback": tb_str,
        }
        print(f"Failed test cached as FAILS[{len(FAILS)}]")
        FAILS.append(failed_test)

        if save_fails:
            fn = Path(f"failed_test_{now}.pkl")
            print(f"Saving failed test to {fn.absolute()}")
            with open(fn, "wb") as fh:
                pickle.dump(failed_test, fh)

In [5]:
n_test = 100

In [6]:
def test_n_stars(n_stars, tolerance=5.0, att_err=5.0, off_nom_roll_max=1.0):
    # Get a random sun-pointed attitude anywhere on the sky for a time in 2024 or 2025
    constraints_all_sky = fa.Constraints(
        off_nom_roll_max=0.0,
        date=CxoTime("2024:001") + np.random.uniform(0, 2) * u.yr,
    )
    att_est = get_random_attitude(constraints_all_sky)

    # Now run find attitude test with this constraint where the test attitude will be
    # randomized consistent with the constraints. This selects stars at an attitude
    # which is randomly displaced from the estimated attitude.
    constraints = fa.Constraints(
        off_nom_roll_max=off_nom_roll_max,
        date=constraints_all_sky.date,
        att=att_est,
        att_err=att_err,
    )
    test_random_all_sky(
        constraints=constraints,
        tolerance=tolerance,
        min_stars=n_stars,
        max_stars=n_stars,
    )

### Test random all-sky with estimated attitude constraint and three stars

Assumptions:

- Three stars available
- Estimated attitude which is accurate to 4 degrees
- True off-nominal < 0.5 degree
- Centroids accurate to 1 arcsec (1-axis 1-sigma)
- Pair distance matching threshold +/- 5.0 arcsec

A few percent of these can fail, but it seems that increasing the tolerance is a 
successful strategy in an estimated attitude constraint.

The current pseudo-random set includes a very-high proper motion star (~6 arcsec/yr)
that poses a challenge.

**No output indicates a successful attitude determination within 1.0 arcsec of
true attitude.**

In [7]:
np.random.seed(0)
for _ in tqdm(range(n_test)):
    test_n_stars(n_stars=3, tolerance=5.0, att_err=5.0, off_nom_roll_max=1.0)

100%|██████████| 100/100 [00:25<00:00,  3.90it/s]


### Test random all-sky with estimated attitude constraint and two stars

Assumptions:

- Two stars available
- Estimated attitude which is accurate to 4 degrees
- True off-nominal < 0.5 degree
- Centroids accurate to 1 arcsec (1-axis 1-sigma)
- Pair distance matching threshold +/- 5.0 arcsec

This is less reliable and the solutions are potentially dubious. Supplying only two
stars currently requires a "code patch" to remind us that we cannot put too much trust
in the solution.

In [8]:
np.random.seed(5)
import find_attitude.find_attitude as fafa

fafa.MIN_STARS_ATT_CONSTRAINT = 2
for _ in tqdm(range(10)):
    test_n_stars(n_stars=2, tolerance=5.0, att_err=5.0, off_nom_roll_max=1.0)
fafa.MIN_STARS_ATT_CONSTRAINT = 3

 30%|███       | 3/10 [00:09<00:18,  2.66s/it]

Failed test cached as FAILS[0]


 60%|██████    | 6/10 [00:27<00:21,  5.32s/it]

Failed test cached as FAILS[1]


 70%|███████   | 7/10 [00:31<00:15,  5.02s/it]

Failed test cached as FAILS[2]


100%|██████████| 10/10 [00:40<00:00,  4.02s/it]


### Random attitude on sky with no constraints

This is similar to the legacy find_attitude except for the off-nominal roll constraint.

- Uses between 4 to 8 stars (randomly selected) for each attitude.
- True off-nominal roll < 1 degree
- Centroids accurate to 1.0 arcsec (1-axis 1-sigma)
- Pair distance matching threshold +/- 3.5 arcsec

In [9]:
np.random.seed(10)
for _ in tqdm(range(n_test // 2)):
    test_random_all_sky(constraints=None, tolerance=3.5)

100%|██████████| 50/50 [02:09<00:00,  2.59s/it]


### Random attitude with pitch near 160 degrees without estimated attitude

- 3 stars available
- True off-nominal < 1 degree
- Centroids accurate to 1.0 arcsec (1-axis 1-sigma)
- Pair distance matching threshold +/- 4.0 arcsec


In [10]:
np.random.seed(20)
for _ in tqdm(range(n_test)):
    constraints = fa.Constraints(
        off_nom_roll_max=1.0,
        date=CxoTime("2024:001") + np.random.uniform(0, 2) * u.yr,
        pitch=160,
        pitch_err=1.5,
    )
    test_random_all_sky(
        constraints=constraints, tolerance=4.0, min_stars=3, max_stars=3
    )

100%|██████████| 100/100 [00:28<00:00,  3.50it/s]


### Debug a failed two-star solution

In [11]:
fafa.MIN_STARS_ATT_CONSTRAINT = 2

fail = FAILS[-1]
sols = fa.find_attitude_solutions(
    fail["stars"],
    tolerance=fail["tolerance"],
    constraints=fail["constraints"],
    log_level="WARNING",
    sherpa_log_level="WARNING",
)

# Get the attitude error (true vs find_attitude solution) in arcsec
for sol in sols:
    d_att = fail["att"].dq(sol["att_fit"])
    sol["ra_dec_roll_err"] = [
        np.round(x * 3600, 2) for x in (d_att.pitch, d_att.yaw, d_att.roll0)
    ]

fafa.MIN_STARS_ATT_CONSTRAINT = 3

In [12]:
# The two solutions include one that is close to the true attitude and one that is
# far away. Check the ra_dec_roll_err for the two solutions.
import pprint

pprint.pprint(sols)

[{'agasc_id_star_map': {36048416: 0, 36438608: 1},
  'agasc_ids': {36048416, 36438608},
  'att_fit': <Quat q1=-0.00744833 q2=-0.86261870 q3=0.50271825 q4=0.05574823>,
  'bad_fit': False,
  'm_yags': array([ 2484.01417237, -1355.1917244 ]),
  'm_zags': array([1520.60813254, 2256.86707141]),
  'ra_dec_roll_err': [7611.78, -4448.23, -10348.51],
  'statval': 2.4176646230805034,
  'summary': <Table masked=True length=2>
AGASC_ID    RA      DEC     YAG    ...  m_zag     dz      dr   m_agasc_id
 int32   float64  float64 float64  ... float64 float64 float64   int64   
-------- -------- ------- -------- ... ------- ------- ------- ----------
35653760 177.4450  2.2317  2485.06 ... 1520.61   -0.34    1.10   36048416
36183880 178.1063  3.0944 -1356.24 ... 2256.87    0.34    1.10   36438608,
  'yags': <Column name='YAG' dtype='float64' length=2>
  2485.058519307462
-1356.2360345987986,
  'zags': <Column name='ZAG' dtype='float64' length=2>
1520.2642480460638
 2257.210820979054},
 {'agasc_id_star_ma

### Real-life NSM recovery 2024:036


#### Start with FFS around 2024:036:02:30:00 with ACA CCD at -1 C. Two stars were found.

In [13]:
att_nsm = [-0.062141268, -0.054177772, 0.920905180, 0.380968348]
date_nsm = "2024:036:03:02:38.343"
slots = [3, 7]
stars = fa.get_stars_from_maude(date_nsm, slots=slots)
stars

slot,YAG,ZAG,MAG_ACA
int64,float64,float64,float64
3,2255.55,1501.58,8.35
7,-2238.57,-1634.20,6.04


In [14]:
constraints = fa.Constraints(
    off_nom_roll_max=1.0,
    date=date_nsm,
    att=att_nsm,
    att_err=5.0,
)

fafa.MIN_STARS_ATT_CONSTRAINT = 2
sols = fa.find_attitude_solutions(
    stars,
    tolerance=2.5,
    constraints=constraints,
    log_level="WARNING",
    sherpa_log_level="WARNING",
)
fafa.MIN_STARS_ATT_CONSTRAINT = 3

In [15]:
# An attitude is found. Do we trust it for much?
sols

[{'yags': <Column name='YAG' dtype='float64' format='.2f' length=2>
  -2238.57
   2255.55,
  'zags': <Column name='ZAG' dtype='float64' format='.2f' length=2>
  -1634.20
   1501.58,
  'm_yags': array([-2237.92042592,  2254.90377841]),
  'm_zags': array([-1633.50580482,  1500.88497425]),
  'att_fit': <Quat q1=-0.11128925 q2=-0.10379516 q3=0.90382672 q4=0.39992314>,
  'statval': 1.8106441183294293,
  'agasc_id_star_map': {638978032: 1, 639119424: 0},
  'agasc_ids': {638978032, 639119424},
  'summary': <Table masked=True length=2>
   slot   YAG      ZAG    MAG_ACA  m_yag   ...  m_zag      dz      dr   m_agasc_id
  int64 float64  float64  float64 float64  ... float64  float64 float64   int64   
  ----- -------- -------- ------- -------- ... -------- ------- ------- ----------
      3  2255.55  1501.58    8.35  2254.90 ...  1500.88    0.70    0.95  639119424
      7 -2238.57 -1634.20    6.04 -2237.92 ... -1633.51   -0.70    0.95  638978032,
  'bad_fit': False}]

#### FFS at around 2024:036:20:30:00 with CCD at -5.4 C and pitch=160

Slots 1 and 2 are junk. Slot 4 is a star but very faint, so let's exclude that.

In [16]:
date_nsm2 = "2024:036:20:31:57.917"
att_nsm2 = [-0.053039170, -0.057369091, 0.921360182, 0.380776903]
slots2 = [0, 3, 5, 6, 7]
stars2 = fa.get_stars_from_maude(date_nsm2, slots=slots2)
stars2

slot,YAG,ZAG,MAG_ACA
int64,float64,float64,float64
0,1758.39,-1362.11,8.45
3,2103.84,-2043.00,9.12
5,706.08,-553.34,9.00
6,-119.61,1939.14,7.25
7,2157.80,-2080.83,8.44


#### Find attitude using only pitch constraint

In [17]:
constraints = fa.Constraints(
    off_nom_roll_max=3.0,
    date=date_nsm2,
    att=att_nsm2,
    att_err=5.0,
)

sols2 = fa.find_attitude_solutions(
    stars2,
    tolerance=2.5,
    constraints=constraints,
    log_level="INFO",
    sherpa_log_level="WARNING",
)

2024-08-07 09:17:52,420 Using AGASC pairs file /Users/aldcroft/ska/data/find_attitude/distances.h5
2024-08-07 09:17:52,427 Starting get_match_graph
2024-08-07 09:17:52,427 Getting matches from file
2024-08-07 09:17:52,551 Adding edges from 371 matching distance pairs
2024-08-07 09:17:52,552 Added total of 89 nodes
2024-08-07 09:17:52,553 Getting all triangles from match graph
2024-08-07 09:17:52,554 Finding triangles that match stars pattern
2024-08-07 09:17:52,554 Checking clique [638201840, 638727896, 638725736, 638724056, 638720752]
2024-08-07 09:17:52,554 Done with graph matching
2024-08-07 09:17:52,555 Found 1 possible AGASC-ID to star index maps
2024-08-07 09:17:52,555 Finding attitude for {638201840: 3, 638727896: 1, 638725736: 0, 638720752: 4, 638724056: 2} stars
2024-08-07 09:17:52,717 Found attitude <Quat q1=-0.06598975 q2=-0.06480277 q3=0.91930096 q4=0.38253327> with statval 5.491245830885055
2024-08-07 09:17:52,718 Adding solution for {638201840, 638720752, 638724056, 63872

In [18]:
sol2 = sols2[0]
sol2["summary"].pprint_all()

slot   YAG     ZAG    MAG_ACA  m_yag    dy   m_zag     dz   dr  m_agasc_id
---- ------- -------- ------- ------- ----- -------- ----- ---- ----------
   0 1758.39 -1362.11    8.45 1758.67 -0.27 -1361.59 -0.51 0.58  638725736
   3 2103.84 -2043.00    9.12 2104.17 -0.33 -2044.75  1.76 1.79  638727896
   5  706.08  -553.34    9.00  706.49 -0.41  -553.52  0.18 0.44  638724056
   6 -119.61  1939.14    7.25 -120.00  0.39  1939.53 -0.40 0.56  638201840
   7 2157.80 -2080.83    8.44 2157.18  0.62 -2079.80 -1.03 1.20  638720752


In [19]:
ska_sun.off_nominal_roll(sol2["att_fit"], time=date_nsm2)

1.7693924862475114

In [21]:
constraints = fa.Constraints(
    off_nom_roll_max=3.0,
    date=date_nsm2,
    pitch=160.0,
    pitch_err=2.0,
)

sols3 = fa.find_attitude_solutions(
    stars2,
    tolerance=2.5,
    constraints=constraints,
    log_level="INFO",
    sherpa_log_level="WARNING",
)

2024-08-07 09:17:52,786 Using AGASC pairs file /Users/aldcroft/ska/data/find_attitude/distances.h5
2024-08-07 09:17:52,802 Starting get_match_graph
2024-08-07 09:17:52,802 Getting matches from file
2024-08-07 09:17:52,932 Adding edges from 1723 matching distance pairs
2024-08-07 09:17:52,935 Added total of 395 nodes
2024-08-07 09:17:52,936 Getting all triangles from match graph
2024-08-07 09:17:52,937 Finding triangles that match stars pattern
2024-08-07 09:17:52,937 Checking clique [638201840, 638727896, 638725736, 638724056, 638720752]
2024-08-07 09:17:52,938 Done with graph matching
2024-08-07 09:17:52,938 Found 1 possible AGASC-ID to star index maps
2024-08-07 09:17:52,939 Finding attitude for {638201840: 3, 638727896: 1, 638725736: 0, 638720752: 4, 638724056: 2} stars
2024-08-07 09:17:53,107 Found attitude <Quat q1=-0.06598975 q2=-0.06480277 q3=0.91930096 q4=0.38253327> with statval 5.491245830885055
2024-08-07 09:17:53,109 Adding solution for {638201840, 638720752, 638724056, 638

In [22]:
sol2 = sols2[0]
sol2["summary"].pprint_all()

slot   YAG     ZAG    MAG_ACA  m_yag    dy   m_zag     dz   dr  m_agasc_id
---- ------- -------- ------- ------- ----- -------- ----- ---- ----------
   0 1758.39 -1362.11    8.45 1758.67 -0.27 -1361.59 -0.51 0.58  638725736
   3 2103.84 -2043.00    9.12 2104.17 -0.33 -2044.75  1.76 1.79  638727896
   5  706.08  -553.34    9.00  706.49 -0.41  -553.52  0.18 0.44  638724056
   6 -119.61  1939.14    7.25 -120.00  0.39  1939.53 -0.40 0.56  638201840
   7 2157.80 -2080.83    8.44 2157.18  0.62 -2079.80 -1.03 1.20  638720752
